In [88]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

In [89]:
data= pd.read_csv('shots.csv')

In [90]:
data=data[['PLAYER_ID','PLAYER_NAME', 'TEAM_ID', 'TEAM_NAME','SHOT_TYPE', 'SHOT_ZONE_BASIC', 'SHOT_ZONE_AREA',
       'SHOT_ZONE_RANGE', 'SHOT_DISTANCE', 'LOC_X', 'LOC_Y',
       'SHOT_ATTEMPTED_FLAG', 'SHOT_MADE_FLAG']]

In [91]:
data["3pa"]=np.where((data["SHOT_DISTANCE"]>=24), 1, 0)
data['2pa'] = np.where((data["SHOT_DISTANCE"]<24), 1, 0)
data["3pm"]=np.where((data["SHOT_MADE_FLAG"]==1) & (data["SHOT_DISTANCE"]>=24), 3, 0)
data['2pm'] = np.where((data["SHOT_MADE_FLAG"]==1) & (data["SHOT_DISTANCE"]<24), 2, 0)
data['Pts']=data["3pm"]+data['2pm']

In [92]:
data=data[["Pts","SHOT_ZONE_BASIC","PLAYER_NAME","PLAYER_ID",'TEAM_NAME']]

In [93]:
data["Mid FGA"]=np.where(data["SHOT_ZONE_BASIC"]=='Mid-Range',1,0)
data["Restricted Area FGA"]=np.where(data["SHOT_ZONE_BASIC"]=='Restricted Area',1,0)
data["Above the Break 3 FGA"]=np.where(data["SHOT_ZONE_BASIC"]=='Above the Break 3',1,0)
data["Right Corner 3 FGA"]=np.where(data["SHOT_ZONE_BASIC"]=='Right Corner 3',1,0)
data['In The Paint (Non-RA) FGA']=np.where(data["SHOT_ZONE_BASIC"]=='In The Paint (Non-RA)',1,0)
data["Left Corner 3 FGA"]=np.where(data["SHOT_ZONE_BASIC"]=='Left Corner 3',1,0)

In [95]:
data=data.sort_values("PLAYER_ID")

In [96]:
x=data[['Mid FGA', 'Restricted Area FGA',
       'Above the Break 3 FGA', 'Right Corner 3 FGA', 'In The Paint (Non-RA) FGA',
       'Left Corner 3 FGA']]
y=data["Pts"]
model_l = sm.OLS(y, x).fit()
print(model_l.summary())
betas = model_l.params
pred_pps=model_l.predict(x).values

                                 OLS Regression Results                                
Dep. Variable:                    Pts   R-squared (uncentered):                   0.461
Model:                            OLS   Adj. R-squared (uncentered):              0.461
Method:                 Least Squares   F-statistic:                          7.083e+04
Date:                Mon, 22 Mar 2021   Prob (F-statistic):                        0.00
Time:                        17:15:07   Log-Likelihood:                     -7.6429e+05
No. Observations:              496182   AIC:                                  1.529e+06
Df Residuals:                  496176   BIC:                                  1.529e+06
Df Model:                           6                                                  
Covariance Type:            nonrobust                                                  
                                coef    std err          t      P>|t|      [0.025      0.975]
--------------------------

In [97]:
df=pd.DataFrame()
for i in data["PLAYER_NAME"].unique():
    

    data_player=data.loc[data["PLAYER_NAME"]=='{}'.format(i)]
    x_data_player=data_player[['Mid FGA', 'Restricted Area FGA',
           'Above the Break 3 FGA', 'Right Corner 3 FGA', 'In The Paint (Non-RA) FGA',
           'Left Corner 3 FGA']]
    y_data_player=data_player["Pts"]
    model_jugador = sm.OLS(y_data_player, x_data_player).fit()
    betas_jugador = model_jugador.params
    df=df.append(betas_jugador,ignore_index=True)

In [99]:
att=data.groupby("PLAYER_ID").sum()

In [100]:
att["FGA"]=att["Mid FGA"]+att["Restricted Area FGA"]+att["Above the Break 3 FGA"]+att["Right Corner 3 FGA"]+att["In The Paint (Non-RA) FGA"]+att["Left Corner 3 FGA"]

In [101]:
att

,Pts,Mid FGA,Restricted Area FGA,Above the Break 3 FGA,Right Corner 3 FGA,In The Paint (Non-RA) FGA,Left Corner 3 FGA,FGA
PLAYER_ID,,,,,,,,
1713,755,88,103,419,52,64,43,769
1717,327,139,5,183,16,27,5,375
2037,412,145,22,165,15,71,22,440
2199,150,1,108,1,0,7,0,117
2200,90,21,36,10,1,24,2,94
...,...,...,...,...,...,...,...,...
1630253,23,2,6,7,3,0,5,23
1630256,269,6,126,37,13,40,15,237
1630264,7,1,2,4,2,0,1,10


In [102]:
player_id=data["PLAYER_ID"].unique()

In [105]:
df=df.set_index(player_id)

In [106]:
df=df.merge(att,left_index=True,right_index=True)

In [110]:
df["Name"]=data["PLAYER_NAME"].unique()